
# CV Screening RAG Chatbot 

This notebook walks you through building a **Retrieval‑Augmented Generation (RAG) chatbot** over a collection of resumes (CVs).  
We will **manually upload CV in PDF** and build a private, local search system.



## 1. Setup & Imports

We’ll use:
- `PyPDF2` → to extract text from CV (PDFs)
- `tiktoken` → for tokenization when chunking text
- `faiss` → local vector database to store embeddings
- `groq api` → for embeddings + chat completions
- `dotenv` → to load API keys from `.env`


In [1]:
%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../requirements.txt'


In [ ]:
import os
import json
import time
from typing import List, Dict
from dataclasses import dataclass

import numpy as np
import faiss
from pypdf import PdfReader
from groq import Groq
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Check API Key
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise RuntimeError("❌ GROQ_API_KEY not set. Please add it to your .env file.")

# Initialize Groq client
client = Groq(api_key=api_key)

print("✅ Groq client ready")

✅ Groq client ready
🔧 Using model: openai/gpt-oss-20b



## 2. Upload & Parse CV (PDFs)

We’ll read CV from a local folder (`./uploads`).  
Each PDF will be converted into plain text for further processing.


In [5]:

UPLOAD_DIR = "uploads"
os.makedirs(UPLOAD_DIR, exist_ok=True)

def extract_text_from_pdf(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

# Parse all resumes in uploads/
resumes = {}
for fname in os.listdir(UPLOAD_DIR):
    if fname.lower().endswith(".pdf"):
        path = os.path.join(UPLOAD_DIR, fname)
        resumes[fname] = extract_text_from_pdf(path)

print(f"✅ Loaded {len(resumes)} resumes")
list(resumes.keys())[:5]


✅ Loaded 0 resumes


[]


## 3. Chunking

Why chunking?  
- pdf can be long. Embedding entire pdf leads to poor retrieval.  
- We split text into **manageable chunks** (300–400 tokens) with slight overlap.

This helps us retrieve only the most relevant pieces.


In [6]:

import tiktoken

def chunk_text(text: str, max_tokens: int = 400, overlap: int = 60) -> List[str]:
    enc = tiktoken.get_encoding("cl100k_base")
    tokens = enc.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk = tokens[i:i+max_tokens]
        chunks.append(enc.decode(chunk))
    return chunks

# Example chunking on first resume
sample_resume = list(resumes.values())[0]
chunks = chunk_text(sample_resume)
print(f"First resume split into {len(chunks)} chunks")
chunks[:2]


IndexError: list index out of range


## 4. Embeddings + FAISS Index

We’ll convert each chunk into a **vector embedding** using OpenAI.  
Then, we’ll store all vectors in a **FAISS index** for fast similarity search.


In [10]:

@dataclass
class DocChunk:
    doc_id: str
    chunk_id: int
    text: str

all_chunks: List[DocChunk] = []
for doc_name, text in resumes.items():
    chunks = chunk_text(text)
    for i, ch in enumerate(chunks):
        all_chunks.append(DocChunk(doc_id=doc_name, chunk_id=i, text=ch))

print(f"Total chunks: {len(all_chunks)}")

def embed_texts(texts: List[str]) -> np.ndarray:
    vectors = []
    for i in range(0, len(texts), 50):
        batch = texts[i:i+50]
        resp = client.embeddings.create(model="text-embedding-3-small", input=batch)
        vectors.extend([d.embedding for d in resp.data])
        time.sleep(0.5)  # be polite
    return np.array(vectors).astype("float32")

# Embed all chunks
texts = [c.text for c in all_chunks]
embeddings = embed_texts(texts)

# Build FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("✅ FAISS index ready")


Total chunks: 23
✅ FAISS index ready



## 5. Retrieval + Answer Generation (RAG)

Workflow:
1. Embed the user’s question
2. Retrieve top‑k most similar chunks
3. Send them along with the question to OpenAI for a **grounded answer**


In [11]:

def search_index(query: str, k: int = 3):
    q_emb = embed_texts([query])
    D, I = index.search(q_emb, k)
    return [(all_chunks[i], float(D[0][j])) for j, i in enumerate(I[0])]

def answer_with_rag(query: str, k: int = 3) -> str:
    results = search_index(query, k)
    context = "\n---\n".join([f"[{r.doc_id}#{r.chunk_id}] {r.text}" for r, _ in results])
    prompt = f"""You are an HR assistant.
Use the following resume excerpts to answer the question:

{context}

Question: {query}
Answer clearly, citing resume IDs like [filename#chunk]."""
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return resp.choices[0].message.content

# Try a sample HR query
print(answer_with_rag("Ranking kandidat berdasarkan pengalaman kerja terlama"))


Berdasarkan pengalaman kerja terlama, berikut adalah peringkat kandidat:

1. **Michael Sihotang** - Pengalaman kerja sebagai Technical Product Ops Intern di Platform Indonesia dan Product Manager Intern di GDP Labs, dengan total pengalaman mencapai lebih dari 4 bulan dari November 2024 hingga sekarang. [CV - Michael Sihotang - Example.pdf#1].

2. **Jessica Enaprilia Sihotang** - Memiliki pengalaman kerja sebagai Project Engineer Intern di PT Iforte Solusi Infotek dari Juli 2024 hingga September 2024, diikuti dengan posisi Operation Intern di Schoters dari Oktober 2024 hingga Januari 2025. Total pengalaman kerja adalah sekitar 4 bulan. [Resume - Jessica Enaprilia Sihotang - Hangry.pdf#0].

3. **Priscilla Auleader Napitupulu** - Terlibat dalam beberapa kompetisi dan mendapatkan penghargaan sejak Maret 2023, tetapi tidak mencantumkan pengalaman kerja secara formal di dalam resume. Tidak ada informasi mengenai pengalaman kerja yang tertera. [CV - Priscilla Auleader Napitupulu.pdf#4].

Jadi


## ✅ Key Takeaways

- RAG lets HR **search CV** using natural language (not just keywords).
- **Chunking + embeddings** improves retrieval accuracy.
- **FAISS** is fast and lightweight for local experiments.

---

## 📌 Next Steps
- Add filters (e.g., years of experience, location).
- Use a managed vector DB (Pinecone, Chroma, pgvector) for scale.
- Improve PDF parsing (layout, tables, OCR).  
- Add guardrails for bias + privacy.
